## 1. 本地版记事本

这是人生第一个python文件。又重新写了一遍。

In [2]:
#_*_coding:utf-8_*_

from os.path import exists
from sys import exit
import time

def main():
    print('''
请输入你的选择：
  1. 打印之前的内容
  2. 写新的内容
  3. 退出
    ''')
    choose = input('>')
    if choose == '1':
        printHistory()
        main()
    elif choose == '2':
        writeNew()
        main()
    elif choose == '3':
        exit(0)
    else:
        print('对不起，我不知道你要干什么。')
        main()

def printHistory():
    if exists('tempfile.txt'):
        print('历史记录：')
        print('='*80)
        txt = open('tempfile.txt')
        lines = txt.readlines()
        for i in lines:
            print(i)
        print('='*80)
    else:
        print('程序第一次运行，还没有历史记录。请选择2录入新内容。')

def writeNew():
    temp = open('tempfile.txt', 'a')
    global content #global声明和赋值不能合一句。
    content = input('今日记录，请输入：')
    temp.write(time.strftime('%d/%m/%Y %H:%M:%S'+'\n')) #在保存当前输入的同时，也保存当前时间。
    temp.write(content+'\n'+'\n')
    temp.close()
    print('''
你是否想把新输入内容存为独立文件？
  1. 随便写写，不用保存。
  2. 保存吧。
    ''')
    c = input('>')  
    if c == '1':
        main()
    elif c == '2':
        saveNew()
    else:
        print('对不起，我不知道你想要干什么。请重新选择是否要存盘。')
        saveNew()

def saveNew():
    print('请输入文件名（扩展名已经默认为txt）：')
    filename = input() + '.txt'
    if exists(filename):
        t = open(filename, 'a')
    else:
        t = open(filename, 'w')
        t.write(filename+'\n')
        t.write('='*80+'\n'+'\n')
    t.write(time.strftime('%d/%m/%Y %H:%M:%S'+'\n'))
    t.write(content+'\n'+'\n')
    t.close() 
        
if __name__ == '__main__':
    main()


请输入你的选择：
  1. 打印之前的内容
  2. 写新的内容
  3. 退出
    
>3


SystemExit: 0

小结：  
1. pass语句对写整体结构很有帮助。这也是之前培炎经常说的。  
2. global语句的声明和赋值不能合为一句，之前没有遇到过。  
3. 在这个例子中，要么函数嵌套，要么用全局变量。似乎没有看到既不嵌套函数也不全局变量的解决方案。  
4. 我之前写的程序，很久以后再运行，往往对输入值的要求不甚明了。例如本例中文件名是否要加扩展名。所以这方面要给足提示。
5. 其实这第一个程序，就已经用了递归。但那时没太注意到，还一直觉得递归很难理解。最近理解后，马上觉得递归太好用了。
6. print仍然是最好用的debug方法。今天把write()的结果赋值给content，要不是打印出来，都不知道为啥是int而不是str。

## 2. Tkinter版本

In [3]:
# _*_coding:utf-8_*_
import tkinter, os, time

root = tkinter.Tk()
root.title('记事本')
root.geometry('640x480')

var = tkinter.StringVar()

def print_history():
    os.chdir(os.path.dirname(os.getcwd()) + '/files')
    if os.path.exists('notebook.txt'):
        file = open(os.getcwd()+'/notebook.txt','r')
        content = file.read()
        '''content = []
        for line in file.readlines(): #如果用这种，会打印出{}
            content.append(line)'''
        file.close()
        os.chdir(os.path.dirname(os.getcwd()) + '/_src')
    else:
        content = '程序第一次运行，还没有历史记录可打印。赶快去输入内容吧。'
    var.set(content)

def save():
    content = entry.get()
    os.chdir(os.path.dirname(os.getcwd()) + '/files')
    if os.path.exists('notebook.txt'):
        file = open(os.getcwd()+'/notebook.txt','a')
    else:
        file = open(os.getcwd()+'/notebook.txt','w')
    file.write(time.strftime('%Y/%m/%d %H:%M:%S'+'\n'))
    file.write(content+'\n'+'\n')
    file.close()
    os.chdir(os.path.dirname(os.getcwd()) + '/_src')

l1 = tkinter.Label(root, text='历史记录区：')
l1.place(x=15, y=10)

label = tkinter.Label(root, textvariable=var, bg='#EEEEEE', width=65, height=18, justify='left',\
                      anchor='nw') #文字左对齐、顶部对齐
label.place(x=15, y=35)

l2 = tkinter.Label(root, text='输入新内容：')
l2.place(x=15, y=370)

entry = tkinter.Entry(root, width=65)
entry.place(x=15, y=395)

b1 = tkinter.Button(root, text='打印历史记录', command=print_history)
b2 = tkinter.Button(root, text='保存输入', command=save)
b1.place(x=150, y=440)
b2.place(x=400, y=440)

root.mainloop()

1. 找不到文件，是因为在文件前多加了个反斜杠\。
2. 打印时要直接read，不要readlines逐句，否则打印{}
3. 可以让save弹出messagebox框，问要不要保存。
4. 如果文字太长，历史记录区就不够了。不知道能不能用滚动条？

## 3. 内网版 ——命令行版 

之前写的是UDP版本，这次决定改成TCP版本。

主要使用socket。这里[[译]Python 中的 Socket 编程（指南）](https://keelii.com/2018/09/24/socket-programming-in-python/)有socket教程。

[Python 网络编程 | 菜鸟教程](https://www.runoob.com/python/python-socket.html)

In [9]:
import socket

HOST = '127.0.0.1'
PORT = 65432

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen()
conn, addr = s.accept()
s.close() # 如果这句话不写，就会提示[Errno 48] Address already in use

OSError: [Errno 48] Address already in use

用ipython已经占据了本地地址，所以改用终端。

In [25]:
import os

open(os.getcwd() + '/nb-client.py', 'w')

<_io.TextIOWrapper name='/Users/caimeijuan/github/myawesome/_src/nb-client.py' mode='w' encoding='UTF-8'>

In [12]:
# _*_coding:utf-8_*_

import socket

HOST = '127.0.0.1'
PORT = 65433

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()

    with conn:
        print('Connected by', addr)
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)

Connected by ('127.0.0.1', 64937)


In [16]:
import socket

HOST = '127.0.0.1'
PORT = 65433

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello world')
    data = s.recv(1024)

print('Received', repr(data))

Received b'Hello world'


In [3]:
# 服务端 
# _*_coding:utf-8_*_
import socket
import time

HOST = '127.0.0.1'
PORT = 65432
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print('Waiting for data...')
    conn, addr = s.accept()
    with conn:
        while True:
            data = conn.recv(1024)
            if not data:
                # 如果没有这句话控制，一旦客户端退出，它就会陷入死循环.
                # if sdata == 'quit'并不能拯救它死循环命运
                break
            conn.sendall(data)
            print('%s: From %s receive: %s \n' % (time.strftime('%Y/%m/%d %H:%M:%S'), str(addr), data.decode('utf-8')))

Waiting for data...
2019/05/09 17:10:38: From ('127.0.0.1', 58743) receive: 试试ipython 



In [4]:
# 客户端
# _*_coding:utf-8_*_

import socket
import sys
import os
# import nbserver
import time


def main():
    HOST = '127.0.0.1'
    PORT = 65432
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOST, PORT))
    while True:
        choose = input('1. 打印历史记录 2. 输入今日记录 （输入quit退出程序）')
        if choose == '1':
            printhistory()
        elif choose == '2':
            data = input('今日记录，请输入（输入quit退出程序）：')
            if data == 'quit':
                sys.exit(0)
            else:
                s.sendall(bytes(data.encode('utf-8')))
                data = s.recv(1024)
                print('Data:', time.strftime('%Y/%m/%d %H:%M:%S'), data.decode('utf-8'))
                save(data.decode('utf-8'))
        elif choose == 'quit':
            sys.exit(0)
        else:
            continue
    s.close()


def printhistory():
    if os.path.exists("tempfile.txt"):
        txt = open("tempfile.txt")
        notelist = txt.readlines()
        print('历史记录：' + '\n')
        for i in notelist:
            print(i)


def save(newline):
    txt = open("tempfile.txt", 'a')
    txt.write(time.strftime('%Y/%m/%d %H:%M:%S') + "\n")
    txt.write(newline + "\n" + "\n")
    txt.close()

if __name__ == '__main__':
    main()

1. 打印历史记录 2. 输入今日记录 （输入quit退出程序）2
今日记录，请输入（输入quit退出程序）：试试ipython
Data: 2019/05/09 17:11:14 试试ipython
1. 打印历史记录 2. 输入今日记录 （输入quit退出程序）quit


SystemExit: 0

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 4. 内网版记事本——浏览器版（web版）

### 4.1 先复习bottle

In [1]:
import bottle

In [24]:
from bottle import route

if __name__ == '__main__':
    main()

@route('/')
def application(environ, start_response):
    start_response('200 OK', [('Content-Type', 'text/html')])
    return ['<h1> Hello world!</h1>']
    
def main():   
    bottle.run(host='localhost', port=8080, app=application)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 12:04:54.344372. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



In [6]:
from bottle import run
import bottle

@bottle.route('/hello')
def hello():
    return "Hello World!"

run(host='localhost', port=8000, debug=True)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 12:14:37.799671. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8000/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/May/2019 12:14:48] "GET /hello HTTP/1.1" 200 12
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/bottle.py:3131: ResourceWarning: unclosed <socket.socket fd=69, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 8000)>
  pass


In [27]:
from bottle import Bottle, run

app = Bottle()

@app.route('/hello')
def hello():
    return 'Hello world!'

run(app, host='localhost', port=8800)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 15:22:06.991513. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8800/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/May/2019 15:22:17] "GET / HTTP/1.1" 404 720
127.0.0.1 - - [10/May/2019 15:22:24] "GET /hello HTTP/1.1" 200 12


In [11]:
from bottle import route, template, run

@route('/')
@route('/hello/<name>')
def hello(name='stranger'):
    return template('Hello, {{name}}. How are you', name=name)

run(host='localhost', port=8000, debug=True)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 12:26:27.102360. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/bottle.py:380: RuntimeWarning: Route <GET /hello/<name>> overwrites a previously defined route
  warnings.warn(msg % (method, rule), RuntimeWarning)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8000/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/May/2019 12:26:30] "GET / HTTP/1.1" 200 28
127.0.0.1 - - [10/May/2019 12:26:37] "GET /hello HTTP/1.1" 200 12
127.0.0.1 - - [10/May/2019 12:27:18] "GET /hello/stranger HTTP/1.1" 200 28
127.0.0.1 - - [10/May/2019 12:27:33] "GET /hello/caimeijuan HTTP/1.1" 200 30
127.0.0.1 - - [10/May/2019 12:27:43] "GET /hello/ahcai HTTP/1.1" 200 25
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/

插入一段装饰器，详细见notebook

In [26]:
import logging, time
def use_logging(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                logging.warning("%s is running" % func.__name__)
                print(time.strftime('%Y-%m-%d %H:%M:%S'))
            elif level == "info":
                logging.info("%s is running" % func.__name__)
                print(time.strftime('%Y-%m-%d %H:%M:%S'))
            return func(*args, **kwargs)
        return wrapper
    return decorator

@use_logging(level="warn")
def foo(name='foo'):
    print("I am %s" % name)
    
foo('ahcai')

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 14:16:58.758376. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


2019-05-10 14:16:58
I am ahcai


In [35]:
from bottle import template

template('I am a {{state}} woman. ', state='Chinese')
template('Hello {{name if name else "world"}}', name=None)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 15:38:23.847537. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


'Hello world'

In [40]:
from bottle import template
# 我压根不知道template干啥用的，所以瞎写这个

def state(state):
    return template('I am a {{state}} woman. ', state=state)
states = ['Chinese', 'English', 'American', 'Russian', 'French']
for i in range(5):
    print(state(states[i]))

I am a Chinese woman. 
I am a English woman. 
I am a American woman. 
I am a Russian woman. 
I am a French woman. 


/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 15:54:12.290430. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [45]:
from bottle import route, run

@route('/object/<id:int>')
def callback(id):
    assert isinstance(id, int)
    return template('{{id}} is {{isinstance(id, int)}} ', id=id)
run(host='localhost', port=8080, debug=True)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 16:20:27.014569. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/bottle.py:380: RuntimeWarning: Route <GET /object/<id:int>> overwrites a previously defined route
  warnings.warn(msg % (method, rule), RuntimeWarning)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8880/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/May/2019 16:20:29] "GET / HTTP/1.1" 200 28
127.0.0.1 - - [10/May/2019 16:20:36] "GET /object/2 HTTP/1.1" 200 10
127.0.0.1 - - [10/May/2019 16:20:40] "GET /object/2int HTTP/1.1" 404 742


In [47]:
from bottle import route, run

@route('/show/<name:re:[a-z]+>')
def callback(name):
    assert name.isalpha()
run(host='localhost', port=8008, debug=True)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 16:25:12.800300. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/bottle.py:380: RuntimeWarning: Route <GET /show/<name:re:[a-z]+>> overwrites a previously defined route
  warnings.warn(msg % (method, rule), RuntimeWarning)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8008/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/May/2019 16:25:21] "GET / HTTP/1.1" 200 28
127.0.0.1 - - [10/May/2019 16:25:30] "GET /show/name HTTP/1.1" 200 0
127.0.0.1 - - [10/May/2019 16:25:36] "GET /show/123 HTTP/1.1" 404 736


使用re都不需要import？？？

### 4.2 一个简单的登录验证界面

In [51]:
from bottle import get, post, request, run

@get('/login')
def login():
    return '''
    <form action="/login" method="post">
            Username: <input name="username" type="text" />
            Password: <input name="password" type="password" />
            <input value="Login" type="submit" />
    </form>
    '''

@post('/login')
def do_login():
    username = request.forms.get('username')
    password = request.forms.get('password')
    if check_login(username, password):
        return "<p>Your login information was correct.</p>"
    else:
        return "<p>Login failed.</p>"
    
def check_login(username, password):
    if username == 'ahcai' and password == '123456':
        return True

run(host='localhost', port=8000, debug=True)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 16:42:57.709357. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8000/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/May/2019 16:43:04] "GET / HTTP/1.1" 200 28
127.0.0.1 - - [10/May/2019 16:43:09] "GET /login HTTP/1.1" 200 232
127.0.0.1 - - [10/May/2019 16:43:18] "POST /login HTTP/1.1" 200 42
127.0.0.1 - - [10/May/2019 16:43:28] "POST /login HTTP/1.1" 200 42
127.0.0.1 - - [10/May/2019 16:43:30] "GET /login HTTP/1.1" 200 232
127.0.0.1 - - [10/May/2019 16:43:34] "POST /login HTTP/1.1" 200 20
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/bottle.py:3131: ResourceWarning: unclosed <socket.socket fd=63, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 8000)>


In [60]:
from bottle import route, run

@route('/hello')
@route('/hello/<name>')
def hello(name='world'):
    return template('hello_template.tpl', name=name)
run(host='localhost', port=8080, debug=True)

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2019-05-10 17:12:21.193839. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/bottle.py:380: RuntimeWarning: Route <GET /hello/<name>> overwrites a previously defined route
  warnings.warn(msg % (method, rule), RuntimeWarning)
Bottle v0.12.16 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [10/May/2019 17:12:29] "GET / HTTP/1.1" 200 28
127.0.0.1 - - [10/May/2019 17:12:37] "GET /hello HTTP/1.1" 200 136
127.0.0.1 - - [10/May/2019 17:12:45] "GET /hello/ahc HTTP/1.1" 200 134
127.0.0.1 - - [10/May/2019 17:12:51] "GET /hello/cai HTTP/1.1" 200 134
/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/bottle.py:3131: ResourceWarning: unclosed <socket.socket fd=69, fam

500错误提示：  

```
Sorry, the requested URL 'http://localhost:8000/hello/ahcai' caused an error:

Template 'hello_template.tpl' not found.
```

造了一个hello_template.tpl文件放在同目录下，运行成功了。  

所以模版文件就是一堆写好的页面，运行时放在同目录下就行了。有点印象了。  

### 4.3 正式的bottle内网版程序

In [1]:
# 服务端程序
# _*_coding:utf-8_*_
import socket
import time
from bottle import route, post, run, template, request
import os


def main():
    global HOST, PORT
    HOST = '127.0.0.1'
    PORT1 = 65429
    PORT2 = 65430
    run(host=HOST, port=PORT1, debug=True, reloader=True)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT2))
        s.listen()
        print('Waiting for data...')
        conn, addr = s.accept()
        with conn:
            while True:
                data = conn.recv(1024)
                if not data:
                    # 如果没有这句话控制，一旦客户端退出，它就会陷入死循环.
                    # if sdata == 'quit'并不能拯救它死循环命运
                    break
                conn.sendall(data)
                print('%s: From %s receive: %s \n' % (time.strftime('%Y/%m/%d %H:%M:%S'), str(addr), data.decode('utf-8')))


@route('/')
def index():
    return template('index', savetxt=False)


@post('/', method='POST')
def save():
    # newline = request.POST.decode('utf-8').get('record')  # 先decode再get，中文无碍
    newline = request.forms.decode('utf-8').get('record')  # 先decode再get，中文无碍
    txt = open("tempfile.txt", 'a')
    txt.write(time.strftime('%Y/%m/%d %H:%M:%S') + "\n")
    txt.write(newline + "\n" + "\n")
    txt.close()
    return template('index', savetxt=True)


@route('/history')
def printhistory():
    if os.path.exists("tempfile.txt"):
        txt = open("tempfile.txt")
        notelist = txt.readlines()
        return template('history', history=notelist)


def save_new(newline):
    txt = open("tempfile.txt", 'a')
    txt.write(time.strftime('%Y/%m/%d %H:%M:%S') + "\n")
    txt.write(newline + "\n" + "\n")
    txt.close()


def print_history():
    if os.path.exists("tempfile.txt"):
        txt = open("tempfile.txt")
        notelist = txt.readlines()
        print('历史记录：' + '\n')
        for i in notelist:
            print(i)


if __name__ == '__main__':
    main()


SystemExit: 1

/Users/caimeijuan/anaconda/envs/python35/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


客户端程序基本不变

## 5 部署于SAE上的bottle程序

In [ ]:
# _*_coding:utf-8_*_
import time
import os
from bottle import run, route, post, template, request


@route('/')
def index():
    return template('index', savetxt=False)


@post('/', method='POST')
def save():
    newline = request.forms.decode('utf-8').get('record')
    txt = open("tempfile.txt", 'a')
    txt.write(time.strftime('%Y/%m/%d %H:%M:%S') + "\n")
    txt.write(newline + "\n" + "\n")
    txt.close()
    return template('index', savetxt=True)


@route('/history')
def printhistory():
    if os.path.exists("tempfile.txt"):
        txt = open("tempfile.txt")
        notelist = txt.readlines()
    else:
        notelist = []
    return template('history', history=notelist)


if __name__ == '__main__':
    global HOST, PORT
    HOST = '0.0.0.0'
    PORT = 5050
    run(host=HOST, port=PORT, debug=True, reloader=True)


比较惊讶的：  

1. bottle可以直接放上去，不用其他程序帮忙。
2. 地址要改成sae要求的“0.0.0.0:5050”
3. sae上可以直接用os建立文本文件来存储，居然不需要数据库也能存储。
4. 每次重新上传程序，就会清空之前的历史文件，卸载我去掉的依赖库。
5. 有几个配套文件设置，记录一下：

```
# Procfile
-------------------------------------------------
web: python main.py runserver 0.0.0.0:5050

# requirements.txt
-------------------------------------------------
bottle

# runtime.txt
-------------------------------------------------
python-3.7.2

``` 

In [1]:
import sae

In [8]:
import sae.kvdb

ModuleNotFoundError: No module named 'sae.kvdb'